In [1]:
import os
import io
from PIL import Image as IM
from IPython.display import display,Image,clear_output
import numpy as np
from scipy.misc import toimage
import keras
from sklearn import preprocessing

Using Theano backend.
Using gpu device 0: GeForce GTX 960 (CNMeM is disabled, CuDNN not available)


In [2]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.models import model_from_json
from keras.utils import np_utils

In [3]:
%matplotlib inline

In [56]:
filelist =[i for i in  os.listdir(os.getcwd()+'\ocrdata') if i.endswith('.png')][5000::4]

In [57]:
len(filelist)

15869

In [58]:
X ,y = [] ,[]
imgpos = [[5,45],[45,85],[85,125],[125,165],[160,200]]
for i in range(len(filelist)):
    clear_output()
    display(i,filelist[i])
    image = IM.open(os.getcwd()+'\ocrdata\%s'%filelist[i])
    imgarray = np.asarray(image)
    for tid in range(5):
        X.append(imgarray[10:50,imgpos[tid][0]:imgpos[tid][1]].reshape(3,40,40))
        y.append(filelist[i][tid])

15868

'ZZZJ7.png'

In [59]:
le = preprocessing.LabelEncoder()
le.fit(y)
y_encode = le.transform(y)
Y = np_utils.to_categorical([[i] for i in y_encode], len(le.classes_))

In [60]:
split = int(len(X)*0.75)

In [61]:
X_train = np.array(X[:split])
Y_train = np.array(Y[:split])
X_test = np.array(X[split:])
Y_test = np.array(Y[split:])

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

In [ ]:
img_channels = 3
img_rows = 40
img_cols = 40
batch_size = 32
nb_classes = len(le.classes_)
nb_epoch = 200

model = Sequential()

model.add(Convolution2D(32, 3, 3, border_mode='same',
                        input_shape=(img_channels, img_rows, img_cols)))
model.add(Activation('relu'))
model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(64, 3, 3, border_mode='same'))
model.add(Activation('relu'))
model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

In [62]:
model = model_from_json(open('cnn_captcha.json').read())
model.load_weights('TWSE_captcha_weights.h5')

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [63]:
batch_size = 32
nb_classes = len(le.classes_)
nb_epoch = 50

In [64]:
model.fit(X_train, Y_train,
              batch_size=batch_size,
              nb_epoch=nb_epoch,
              validation_data=(X_test, Y_test),
              shuffle=True)

Train on 59508 samples, validate on 19837 samples
Epoch 1/50
59508/59508 [==============================] - 102s - loss: 0.0769 - acc: 0.9808 - val_loss: 0.0580 - val_acc: 0.9898
Epoch 2/50
59508/59508 [==============================] - 102s - loss: 0.0552 - acc: 0.9870 - val_loss: 0.0634 - val_acc: 0.9879
Epoch 3/50
59508/59508 [==============================] - 101s - loss: 0.0467 - acc: 0.9892 - val_loss: 0.0502 - val_acc: 0.9916
Epoch 4/50
59508/59508 [==============================] - 102s - loss: 0.0409 - acc: 0.9908 - val_loss: 0.0446 - val_acc: 0.9935
Epoch 5/50
59508/59508 [==============================] - 102s - loss: 0.0370 - acc: 0.9923 - val_loss: 0.0585 - val_acc: 0.9903
Epoch 6/50
59508/59508 [==============================] - 102s - loss: 0.0348 - acc: 0.9928 - val_loss: 0.0431 - val_acc: 0.9941
Epoch 7/50
59508/59508 [==============================] - 101s - loss: 0.0321 - acc: 0.9933 - val_loss: 0.0476 - val_acc: 0.9932
Epoch 8/50
59508/59508 [=======================

In [65]:
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Test score: 0.056824512657
Test accuracy: 0.993194535464


In [66]:
json_string = model.to_json()
open('cnn_captcha.json', 'w').write(json_string)
model.save_weights('TWSE_captcha_weights.h5')

[WARNING] TWSE_captcha_weights.h5 already exists - overwrite? [y/n]y
[TIP] Next time specify overwrite=True in save_weights!
